In [ ]:
from nb_shared import *

### Cluster for batch processing

In [ ]:
cluster = start_cluster(n_workers=20, local=False)

## Snow-17 SWE

s17_swe = Snow17SWE(
    SWE_HOME_DIR + 'CBRFC_Virgin_1981_2020_SWE_inches.csv',
)

target_zones = list(s17_swe.csv.columns)
target_zones

In [ ]:
target_ch5ids = [
    'ANBC2',
    'RCYC2',
    'FPTC2',
    'ENMC2',
    'WBRW4',
    'BPPA3',
    'OAWU1'
]

## SWANN for CBRFC zone

In [ ]:
zone_db = ZoneDB(DB_CONNECTION_INFO)
swann_xr = xr.open_mfdataset(SWANN_HOME_DIR + '*.nc', parallel=True)

# Get peak SWE for each year

In [ ]:
target_zones = zone_db.zone_in_ch5_id(target_ch5ids)
target_zone_names = [cz.zone for cz in target_zones]

In [ ]:
peak_swe = peak_swe_for_swann(swann_xr, target_zone_names, zone_db) 

In [ ]:
peak_swe['BPPA3HLH'].hvplot()

In [ ]:
%%time
swann_swe = {
    zone: swann_swe_for_zone(swann_xr, zone, zone_db) for zone in target_zone_names
}

plot = compare_peak_swe(
    peak_swe_for_pd(s17_swe.csv), 
    peak_swe_for_swann(swann_xr, target_zones, zone_db) 
)

In [ ]:
swann_swe['BPPA3HLH'].hvplot()

In [ ]:
ZonePlotter.show_all(plot, 1)

In [ ]:
ZonePlotter.save_html(HTML_OUTPUT + 'virgin_peak_SWE', plot, 1)

In [ ]:
cluster.shutdown()